# Make an example MTH5 file

This notebook will demonstrate how you can make an MTH5 file.

In [2]:
# this is a hack for now, once we have a real package with an install this should be removed.
import os
from pathlib import Path
import numpy as np

# change this to your local mth5 repository
os.chdir(r"c:\Users\jpeacock\Documents\GitHub\mth5")

# import mth5 metadata
from mth5 import mth5, metadata, timeseries

## Initialize MTH5 file

In [6]:
mth5_obj = mth5.MTH5()
mth5_obj.open_mth5(Path(Path.cwd(), "example.h5"))
mth5_obj

2020-12-04T14:47:31 [line 513] mth5.mth5.MTH5._initialize_file - INFO: Initialized MTH5 file c:\Users\jpeacock\Documents\GitHub\mth5\example.h5 in mode 'w'


/:
    |- Group: Survey
    ----------------
        |- Group: Filters
        -----------------
            --> Dataset: summary
            ......................
        |- Group: Reports
        -----------------
            --> Dataset: summary
            ......................
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Stations
        ------------------
            --> Dataset: summary
            ......................

## Add a new station

In [7]:
new_station = mth5_obj.add_station("mt001")
new_station

/Survey/Stations/mt001:
    --> Dataset: summary
    ......................

In [8]:
new_station.metadata.id

'mt001'

### Update Station metadata

In [9]:
new_station.metadata.time_period.start = "2020-01-01T12:00:00"
new_station.metadata.time_period.end = "2020-01-05T14:00:00"
new_station.metadata.location.latitude = 40.0
new_station.metadata.location.longitude = -120
new_station.metadata.location.elevation = 500
new_station.metadata

{
    "station": {
        "acquired_by.author": null,
        "channels_recorded": [],
        "data_type": null,
        "geographic_name": null,
        "hdf5_reference": "<HDF5 object reference>",
        "id": "mt001",
        "location.declination.model": null,
        "location.declination.value": null,
        "location.elevation": 500.0,
        "location.latitude": 40.0,
        "location.longitude": -120.0,
        "mth5_type": "Station",
        "orientation.method": null,
        "orientation.reference_frame": "geographic",
        "provenance.creation_time": "2020-12-04T22:47:34.371165+00:00",
        "provenance.software.author": null,
        "provenance.software.name": null,
        "provenance.software.version": null,
        "provenance.submitter.author": null,
        "provenance.submitter.email": null,
        "provenance.submitter.organization": null,
        "time_period.end": "2020-01-05T14:00:00+00:00",
        "time_period.start": "2020-01-01T12:00:00+00:00"
 

### Add Run

In [10]:
new_run = new_station.add_run("mt001a")
new_run.metadata.sample_rate = 256
new_run.metadata.time_period.start = "2020-01-01T12:00:00"
new_run.metadata

{
    "run": {
        "acquired_by.author": null,
        "channels_recorded_auxiliary": null,
        "channels_recorded_electric": null,
        "channels_recorded_magnetic": null,
        "data_logger.firmware.author": null,
        "data_logger.firmware.name": null,
        "data_logger.firmware.version": null,
        "data_logger.id": null,
        "data_logger.manufacturer": null,
        "data_logger.timing_system.drift": null,
        "data_logger.timing_system.type": null,
        "data_logger.timing_system.uncertainty": null,
        "data_logger.type": null,
        "data_type": null,
        "hdf5_reference": "<HDF5 object reference>",
        "id": "mt001a",
        "metadata_by.author": null,
        "mth5_type": "Run",
        "sample_rate": 256.0,
        "time_period.end": "1980-01-01T00:00:00+00:00",
        "time_period.start": "2020-01-01T12:00:00+00:00"
    }
}

### Add channels to the run

In [11]:
for index, component in enumerate(["ex", "ey"], 1):
    e_metadata = metadata.Electric(**{"time_period.start": "2020-01-01T12:00:00",
                                      "time_period.end": "2020-01-02T12:00:00",
                                      "sample_rate": 256,
                                      "component": component,
                                      "channel_number": index,
                                      "dipole_length": 100,
                                      "measurement_azimuth": 0})
    new_e = new_run.add_channel(component, 
                                 channel_type="electric",
                                 data=np.random.rand(4096),
                                 channel_metadata=e_metadata)

for index, component in enumerate(["hx", "hy", "hz"], 3):
    h_metadata = metadata.Magnetic(**{"time_period.start": "2020-01-01T12:00:00",
                        "time_period.end": "2020-01-02T12:00:00",
                        "sample_rate": 256,
                        "component": component,
                        "channel_number": index,
                        "measurement_azimuth": 0})
    new_h = new_run.add_channel(component, 
                                 channel_type="magnetic", 
                                 data=np.random.rand(4096),
                                 channel_metadata=h_metadata)

#### Check the summary table to make sure everything is there

In [12]:
new_run.summary_table

index | component |            start             |             end              | n_samples | measurement_type |  units  |     hdf5_reference     
--------------------------------------------------------------------------------------------------------------------------------------------------
  0   |  ex   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  electric   |  None   | <HDF5 object reference>
  1   |  ey   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  electric   |  None   | <HDF5 object reference>
  2   |  hx   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  magnetic   |  None   | <HDF5 object reference>
  3   |  hy   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  magnetic   |  None   | <HDF5 object reference>
  4   |  hz   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  magnetic   |  None   | <HDF5 object reference>

#### Check the summary table of the stations_group

In [13]:
mth5_obj.stations_group.summary_table

index | station  |    run    | latitude | longitude | elevation | component |            start             |             end              | n_samples | measurement_type | azimuth | tilt |  units  |     hdf5_reference     
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
  0   |  mt001   |  mt001a   | 0.0 | 0.0 | 0.0 |  ex   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  electric   | 0.0 | 0.0 |  None   | <HDF5 object reference>
  1   |  mt001   |  mt001a   | 0.0 | 0.0 | 0.0 |  ey   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  electric   | 0.0 | 0.0 |  None   | <HDF5 object reference>
  2   |  mt001   |  mt001a   | 0.0 | 0.0 | 0.0 |  hx   |  2020-01-01T12:00:00+00:00   |  2020-01-02T12:00:00+00:00   | 4096 |  magnetic   | 0.0 | 0.0 |  None   | <HDF5 object reference

In [14]:
mth5_obj

/:
    |- Group: Survey
    ----------------
        |- Group: Filters
        -----------------
            --> Dataset: summary
            ......................
        |- Group: Reports
        -----------------
            --> Dataset: summary
            ......................
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Stations
        ------------------
            |- Group: mt001
            ---------------
                |- Group: mt001a
                ----------------
                    --> Dataset: ex
                    .................
                    --> Dataset: ey
                    .................
                    --> Dataset: hx
                    .................
                    --> Dataset: hy
                    .................
                    --> Dataset: hz
                    .................
                    --> Dataset: summary
    

## Add a channel from the time series object

This is the common way that time series will actually be added to the MTH5 file.  Data will be read in from a file to create an mth5.timeseries.RunTS object which can then put into a run or channel

In [3]:
channel_ts_list = []
station_metadata = metadata.Station(id="mt001")
run_metadata = metadata.Run()
run_metadata.id = "mt001b"
run_metadata.sample_rate = 256
run_metadata.time_period.start = "2020-01-01T12:00:00"

# make electric channels
for index, component in enumerate(["ex", "ey"], 1):
    e_metadata = metadata.Electric(**{"time_period.start": "2020-01-01T12:00:00",
                                      "sample_rate": 256,
                                      "component": component,
                                      "channel_number": index,
                                      "dipole_length": 100,
                                      "measurement_azimuth": 0})
    ch = timeseries.ChannelTS(channel_type="electric",
                             data=np.random.rand(4096),
                             channel_metadata=e_metadata,
                             station_metadata=station_metadata,
                             run_metadata=run_metadata)
    channel_ts_list.append(ch)
    
# make magnetic channels
for index, component in enumerate(["hx", "hy", "hz"], 3):
    h_metadata = metadata.Magnetic(**{"time_period.start": "2020-01-01T12:00:00",
                                      "sample_rate": 256,
                                      "component": component,
                                      "channel_number": index,
                                      "measurement_azimuth": 0})
    ch = timeseries.ChannelTS(channel_type="magnetic",
                             data=np.random.rand(4096),
                             channel_metadata=h_metadata,
                             station_metadata=station_metadata,
                             run_metadata=run_metadata)
    channel_ts_list.append(ch)
    


In [4]:
# create a timeseries.RunTS object
run_obj = timeseries.RunTS(array_list=channel_ts_list)

2020-12-04T14:46:45 [line 846] mth5.timeseries.RunTS.validate_metadata - WARNING: sample rate of dataset 256.0 does not match metadata sample rate None updating metatdata value to 256.0
2020-12-04T14:46:45 [line 856] mth5.timeseries.RunTS.validate_metadata - WARNING: start time of dataset 2020-01-01T12:00:00+00:00 does not match metadata start 1980-01-01T00:00:00+00:00 updating metatdata value to 2020-01-01T12:00:00+00:00
2020-12-04T14:46:45 [line 866] mth5.timeseries.RunTS.validate_metadata - WARNING: end time of dataset 2020-01-01T12:00:15.996093+00:00 does not match metadata end 1980-01-01T00:00:00+00:00 updating metatdata value to 2020-01-01T12:00:15.996093+00:00


In [15]:
new_run_02 = new_station.add_run("mt001b")
new_run_02.from_runts(run_obj)

[Channel Electric:
 -------------------
 	component:        ex
 	data type:        electric
 	data format:      float64
 	data shape:       (4096,)
 	start:            2020-01-01T12:00:00+00:00
 	end:              2020-01-01T12:00:15.996093+00:00
 	sample rate:      256.0,
 Channel Electric:
 -------------------
 	component:        ey
 	data type:        electric
 	data format:      float64
 	data shape:       (4096,)
 	start:            2020-01-01T12:00:00+00:00
 	end:              2020-01-01T12:00:15.996093+00:00
 	sample rate:      256.0,
 Channel Magnetic:
 -------------------
 	component:        hx
 	data type:        magnetic
 	data format:      float64
 	data shape:       (4096,)
 	start:            2020-01-01T12:00:00+00:00
 	end:              2020-01-01T12:00:15.996093+00:00
 	sample rate:      256.0,
 Channel Magnetic:
 -------------------
 	component:        hy
 	data type:        magnetic
 	data format:      float64
 	data shape:       (4096,)
 	start:            2020-01-01